In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import functools
from datetime import datetime, timedelta

import requests
import pandas as pd
from dateutil.parser import parse as parse_dt

from split_query.expressions import Attribute, Ge, Le, Gt, Lt, And, Or, Eq, In, Not, math_repr
from interface import DataSet
from split_query.converters import convert_expression, soql, in_to_or, map_to_ids
from cache import CachingBackend, simplify

In [ ]:
class SoQLError(Exception):
    pass

def soda_query(domain, endpoint, **params):
    # Formatting of domain, endpoint and parameters
    url = 'https://{0}/resource/{1}.json'.format(domain, endpoint)
    params = {'$' + key: value for key, value in params.items()}
    response = requests.get(url=url, params=params)
    data = response.json()
    if response.status_code == 200:
        return data
    else:
        if 'message' in data:
            raise SoQLError(data['message'])
        elif 'data' in data:
            raise SoQLError('SoQL query error: {query}'.format(**data['data']))
        elif 'code' in data:
            raise SoQLError('SoQL error code: {code}'.format(**data))
        else:
            raise SoQLError('SoQL unknown error: {}'.format(data))

In [ ]:
def _map_to_year(obj):
    ''' Map a datetime query to numeric year only. '''
    if (isinstance(obj, Ge) or isinstance(obj, Gt)) and obj.attribute == Attribute('datetime'):
        return Ge(Attribute('year'), obj.value.year)
    if (isinstance(obj, Le) or isinstance(obj, Lt)) and obj.attribute == Attribute('datetime'):
        return Le(Attribute('year'), obj.value.year)
    return obj

def _widen_to_year(obj):
    if isinstance(obj, Ge) and obj.attribute == Attribute('datetime'):
        return Ge(Attribute('datetime'), datetime(obj.value.year, 1, 1))
    if isinstance(obj, Le) and obj.attribute == Attribute('datetime'):
        return Lt(Attribute('datetime'), datetime(obj.value.year + 1, 1, 1))
    return obj

def get_actual_query(expression):
    return convert_expression(expression, hook=_widen_to_year)

def get_soql_where(expression):
    expression = convert_expression(expression, hook=_map_to_year)
    expression = simplify(expression)
    expression = convert_expression(expression, hook=in_to_or)
    return convert_expression(expression, hook=soql)

def parse_remote(entry):
    return {
        'datetime': parse_dt(entry['daet_time']),
        'hourly_count': int(entry['qv_market_peel_st']),
        'sensor_id': int(entry['sensor_id'])}

class PedestrianRemote(object):

    def __init__(self):
        self.get = functools.partial(soda_query, 'data.melbourne.vic.gov.au', 'cb85-mn2u')
        self.page = 50000

    def paged_query(self, where):
        offset = 0
        while True:
            part = self.get(where=where, limit=self.page, offset=offset)
            for entry in part:
                yield entry
            if len(part) < self.page:
                break
            offset += self.page

    def query(self, expression):
        actual = get_actual_query(expression)
        where = get_soql_where(expression)
        logging.info('REMOTE: ' + where)
        result = pd.DataFrame(list(map(parse_remote, self.paged_query(where))))
        return actual, result

    def mock_data(self):
        return pd.DataFrame(dict(
            datetime=[datetime(2017, 3, 2) + timedelta(hours=1) * (i + 3) for i in range(4)],
            hourly_count=[244, 543, 322, 100],
            sensor_id=[27, 27, 27, 27]))

    def estimate_count(self, expr):
        ''' This appears to be called twice on repr?? '''
        expr = simplify(expr)
        if isinstance(expr, And):
            lower = next(cl for cl in expr.clauses if isinstance(cl, Ge) or isinstance(cl, Gt)).value
            upper = next(cl for cl in expr.clauses if isinstance(cl, Le) or isinstance(cl, Lt)).value
            count = len(next(cl for cl in expr.clauses if isinstance(cl, In)).valueset)
        return int((upper - lower).total_seconds() // 3600) * count

In [ ]:
backend = CachingBackend(PedestrianRemote())
attributes = [Attribute(n) for n in ['datetime', 'hourly_count', 'sensor_id']]
desc = 'Hourly pedestrian counts from various intersections in Melbourne.'
dataset = DataSet('Pedestrians', attributes, backend, description=desc)

def show_result(filtered):
    g = filtered.get().groupby('sensor_id')
    return pd.DataFrame(dict(
        min_dt=g.datetime.min(),
        max_dt=g.datetime.max(),
        count=g.datetime.count()))[['min_dt', 'max_dt', 'count']]

In [ ]:
dataset

In [ ]:
filtered = dataset[
    dataset.datetime.between(datetime(2015, 5, 3), datetime(2016, 2, 3)) &
    dataset['sensor_id'].isin([27, 28])]
filtered

In [ ]:
print('Records: {}'.format(filtered.get().shape[0]))
filtered.get().head()

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2015, 5, 3), datetime(2015, 10, 3)) &
    dataset.sensor_id.isin([27])])

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2015, 5, 3), datetime(2016, 2, 3)) &
    dataset.sensor_id.isin([27, 28])])

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2015, 2, 3), datetime(2015, 10, 7)) &
    dataset.sensor_id.isin([27, 28])])

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2015, 2, 3), datetime(2015, 10, 7)) &
    dataset.sensor_id.isin([27, 28, 29])])

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2015, 2, 3), datetime(2016, 10, 7)) &
    dataset.sensor_id.isin([27, 28, 29])])

In [ ]:
show_result(dataset[
    dataset.datetime.between(datetime(2016, 2, 3), datetime(2016, 10, 7)) &
    dataset.sensor_id.isin([27, 28, 29])])

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize': (15, 8)})

In [ ]:
dataset[
    dataset.datetime.between(datetime(2016, 11, 1), datetime(2016, 11, 15)) &
    dataset.sensor_id.isin([28, 29])].get().set_index(['datetime', 'sensor_id']).unstack().plot();